## Applied Data Science Capstone
# Segmenting and Clustering Neighborhoods in Toronto
### Xavier Martinez
# Assignment Week 3 - Exercise 1

In [1]:
# IMPORT THE REQUIRED LIBRARIES

#import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis working with dataframes
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

#import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
#import matplotlib.cm as cm
#import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

!pip install lxml
from lxml import html

print('Libraries imported')

     |████████████████████████████████| 5.8MB 8.4MB/s eta 0:00:01
Libraries imported


In [2]:
# padas is used for scrapping the wikipedia website information
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Toronto_PC = pd.read_html(url)

# check the data type of the information and content obtained
print(type(Toronto_PC))
Toronto_PC  

<class 'list'>


[    Postcode           Borough           Neighborhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 282      M8Z         Etobicoke              Mimico NW
 283      M8Z         Etobicoke     The Queensway West
 284      M8Z         Etobicoke  Royal York South West
 285      M8Z         Etobicoke         South of Bloor
 286      M9Z      Not assigned           Not assigned
 
 [287 rows x 3 columns],
                                                   0   \
 0                                                NaN   
 1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
 2                                                 NL   
 3                                                  A   
 
                          

In [3]:
# The information scrapped is stored as a list, from which the first item corresponds to the postcodes table
# this first element is passed to a dataframe

Toronto_pc_df=pd.DataFrame(Toronto_PC[0])
print(type(Toronto_pc_df))
print(Toronto_pc_df.shape)
Toronto_pc_df.head()

<class 'pandas.core.frame.DataFrame'>
(287, 3)


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# The registers with Borough not assigned are removed
Toronto_pc_df=Toronto_pc_df[Toronto_pc_df.Borough !='Not assigned']
print(Toronto_pc_df.shape)
Toronto_pc_df.head()

(210, 3)


,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [5]:
# The amount of different postcodes is obtained by creating a set of them. 
# This has to be the number of rows of the final dataframe. 

print("The df contains ",len(set(Toronto_pc_df.Postcode))," different postcodes")

The df contains  103  different postcodes


In [6]:
# A new dataframe will be created with the clean data
#del Toronto_clean_df   # case this cell has to be run multiple time, this line has to be de-commentarised

# define the dataframe columns
column_names = ['Postcode','Borough', 'Neighborhood'] 
# instantiate the dataframe
Toronto_clean_df = pd.DataFrame(columns=column_names)

In [7]:
# a loop over the existing data is run to group and compleate the information
# neighborhoods will be stored in a list
# not assigned neighborhoods will be populated with the borough name

T_clean_pc_set=[]
T_clean_neigh_set=[]
populated_neighb=0
for i in range(len(Toronto_pc_df)):
    neighborhood_list=[]
    pc=Toronto_pc_df.iloc[i].Postcode
    if pc not in T_clean_pc_set:       #  here I'll check if the pc has already been transfered to the clean df, if so loop
        borough=Toronto_pc_df.iloc[i].Borough
        neighborhood=Toronto_pc_df.iloc[i].Neighborhood
        if neighborhood=="Not assigned": 
            neighborhood=borough 
            populated_neighb=populated_neighb+1
        neighborhood_list.append(neighborhood)

        for j in range(i+1,len(Toronto_pc_df)):
            if Toronto_pc_df.iloc[j].Postcode==pc:
                if Toronto_pc_df.iloc[j].Borough != borough:
                    print ("carefull, iloc: ",j," Postcode: ", Toronto_pc_df.iloc[j].Borough," differs from the exected: ",borough)
                elif Toronto_pc_df.iloc[j].Neighborhood not in neighborhood_list:
                    neighborhood_list.append(Toronto_pc_df.iloc[j].Neighborhood)
                
        Toronto_clean_df= Toronto_clean_df.append({'Postcode':pc,
                                          'Borough': borough,
                                          'Neighborhood': neighborhood_list}, ignore_index=True)
        T_clean_pc_set=set(Toronto_clean_df.Postcode)

# Check that the amount of information has no chanded with the regrouping. 
# Number of neighborhoods could have increased to to populat with borough name when was not assigned

print("The number of neighborhoods populated with the borough name is: ",populated_neighb)
print("The clean df contains ",len(set(Toronto_clean_df.Postcode))," different postcodes")
print("The clean df contains ",len(set(Toronto_clean_df.Borough))," different boroughs")
print("The clean df contains ",len(set(Toronto_clean_df.Neighborhood.sum()))," different neighborhood")

print("The df shape is: ",Toronto_clean_df.shape)
Toronto_clean_df

The number of neighborhoods populated with the borough name is:  1
The clean df contains  103  different postcodes
The clean df contains  11  different boroughs
The clean df contains  207  different neighborhood
The df shape is:  (103, 3)


,Postcode,Borough,Neighborhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,[Harbourfront]
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]"
4,M7A,Downtown Toronto,[Queen's Park]
...,...,...,...
98,M8X,Etobicoke,"[The Kingsway, Montgomery Road, Old Mill North]"
99,M4Y,Downtown Toronto,[Church and Wellesley]
100,M7Y,East Toronto,[Business Reply Mail Processing Centre 969 Eas...
101,M8Y,Etobicoke,"[Humber Bay, King's Mill Park, Kingsway Park S..."


# Assignment Week 3 - Exercise 2

In [8]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 7.0MB/s ta 0:00:011
     |████████████████████████████████| 829kB 19.3MB/s eta 0:00:01
     |████████████████████████████████| 81kB 12.8MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [ ]:
# GEOCODER STAYS IRRESPONSIVE, THEREFORE THIS CODE IS NO EXECUTED AND THE CSV FILE USED INSTEAD.

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
for i in range(len(Toronto_pc_df)):
    pc=Toronto_pc_df.iloc[i].Postcode
    print ("fetching coords for: ",pc)
    iteration=0
    while(lat_lng_coords is None):
        iteration=iteration+1
        print("iteration: {}".format(iteration), end="\r", flush=True)
        g = geocoder.google('{}, Toronto, Ontario'.format(pc))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    Toronto_clean_df.Latitude[i]=latitude
    Toronto_clean_df.Longitude[i]=longitude
    
print("The df shape is: ",Toronto_clean_df.shape)
Toronto_clean_df

In [9]:
# The Post Code coordinates are read from the csv file
T_pc_latlong_df=pd.read_csv('Geospatial_Coordinates.csv')
T_pc_latlong_df.rename(columns={'Postal Code':'Postcode'},inplace=True)
T_pc_latlong_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# Preparation of the clean dataframe, adding the Latitude and Longitude columns
Toronto_clean_df['Latitude'] = 0.00
Toronto_clean_df['Longitude'] = 0.00
Toronto_clean_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],0.0,0.0
1,M4A,North York,[Victoria Village],0.0,0.0
2,M5A,Downtown Toronto,[Harbourfront],0.0,0.0
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]",0.0,0.0
4,M7A,Downtown Toronto,[Queen's Park],0.0,0.0


In [11]:
# The coordinates information is added to the previous clean data (Toronto_clean_df)
# note that the Postcode at both dataframes may not be in the same order
for i in range(len(Toronto_pc_df)):
    pc=Toronto_pc_df.iloc[i].Postcode
    found=0
    total_found=0
#    print ("fetching coords for: ",pc)
    for j in range(len(T_pc_latlong_df)):
        if T_pc_latlong_df.Postcode[j]==pc:
            latitude = T_pc_latlong_df.Latitude[j]
            longitude = T_pc_latlong_df.Longitude[j]
            found=1
            total_found=total_found+1
        if found==1:
            break
        elif j==(len(T_pc_latlong_df)):
            print("pc not found")
            latitude = 0.0
            longitude = 0.0
    Toronto_clean_df.Latitude[i]=latitude
    Toronto_clean_df.Longitude[i]=longitude
print("Todal coordinates linked to pc:",total_found)    
print("The df shape is: ",Toronto_clean_df.shape)
Toronto_clean_df

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Todal coordinates linked to pc: 1
The df shape is:  (103, 5)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.753259,-79.329656
1,M4A,North York,[Victoria Village],43.725882,-79.315572
2,M5A,Downtown Toronto,[Harbourfront],43.654260,-79.360636
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]",43.718518,-79.464763
4,M7A,Downtown Toronto,[Queen's Park],43.718518,-79.464763
...,...,...,...,...,...
98,M8X,Etobicoke,"[The Kingsway, Montgomery Road, Old Mill North]",43.659526,-79.340923
99,M4Y,Downtown Toronto,[Church and Wellesley],43.733283,-79.419750
100,M7Y,East Toronto,[Business Reply Mail Processing Centre 969 Eas...,43.733283,-79.419750
101,M8Y,Etobicoke,"[Humber Bay, King's Mill Park, Kingsway Park S...",43.691116,-79.476013


# Assignment Week 3 - Exercise 3

In [12]:
# IMPORT THE REQUIRED LIBRARIES

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis working with dataframes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    sci

In [13]:
address = 'Toronto'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates for {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinates for Toronto are 43.653963, -79.387207.


In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_clean_df['Latitude'], Toronto_clean_df['Longitude'], Toronto_clean_df['Borough'], Toronto_clean_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [16]:
# Now will proceed to analyse the neighborhoods using FourSquare
# The FourSquarre credentials are stored in a txt file to prevent them being shown in the code
Credentials = open('FourSquare_credentials.txt', 'r')
CLIENT_ID = Credentials.read(48)
CLIENT_SECRET = Credentials.read(48) 

VERSION = '20180605' # Foursquare API version

In [32]:
# this function return the venues nearby a certain location
def getNearbyVenues(pc,names, latitudes, longitudes, radius=500,LIMIT=100):
    
    venues_list=[]
    counter=1
    for pc, name, lat, lng in zip(pc,names, latitudes, longitudes):
        print(counter,pc,name)
        counter=counter+1  
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            pc,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
# The defined function is applied to all neighborhoods
toronto_venues = getNearbyVenues(pc=Toronto_clean_df['Postcode'],
                                 names=Toronto_clean_df['Neighborhood'],
                                 latitudes=Toronto_clean_df['Latitude'],
                                 longitudes=Toronto_clean_df['Longitude']
                                 )

1 M3A ['Parkwoods']
2 M4A ['Victoria Village']
3 M5A ['Harbourfront']
4 M6A ['Lawrence Heights', 'Lawrence Manor']
5 M7A ["Queen's Park"]
6 M9A ["Queen's Park"]
7 M1B ['Rouge', 'Malvern']
8 M3B ['Don Mills North']
9 M4B ['Woodbine Gardens', 'Parkview Hill']
10 M5B ['Ryerson', 'Garden District']
11 M6B ['Glencairn']
12 M9B ['Cloverdale', 'Islington', 'Martin Grove', 'Princess Gardens', 'West Deane Park']
13 M1C ['Highland Creek', 'Rouge Hill', 'Port Union']
14 M3C ['Flemingdon Park', 'Don Mills South']
15 M4C ['Woodbine Heights']
16 M5C ['St. James Town']
17 M6C ['Humewood-Cedarvale']
18 M9C ['Bloordale Gardens', 'Eringate', 'Markland Wood', 'Old Burnhamthorpe']
19 M1E ['Guildwood', 'Morningside', 'West Hill']
20 M4E ['The Beaches']
21 M5E ['Berczy Park']
22 M6E ['Caledonia-Fairbanks']
23 M1G ['Woburn']
24 M4G ['Leaside']
25 M5G ['Central Bay Street']
26 M6G ['Christie']
27 M1H ['Cedarbrae']
28 M2H ['Hillcrest Village']
29 M3H ['Bathurst Manor', 'Downsview North', 'Wilson Heights']
30 M

In [34]:
# Check the size of the data gathered, and how the dataframe looks like 
print(toronto_venues.shape)
toronto_venues.head()

(2579, 8)


,Postcode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,[Parkwoods],43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,[Parkwoods],43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,[Victoria Village],43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,[Victoria Village],43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,M4A,[Victoria Village],43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [35]:
# Check the number of data gathered for each neighborhood 
toronto_venues.groupby('Postcode').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,,
M1C,100,100,100,100,100,100,100
M1E,1,1,1,1,1,1,1
M1G,1,1,1,1,1,1,1
M1H,100,100,100,100,100,100,100
M1J,7,7,7,7,7,7,7
M1K,3,3,3,3,3,3,3
M1L,20,20,20,20,20,20,20
M1M,16,16,16,16,16,16,16
M1N,5,5,5,5,5,5,5


In [36]:
# Check the number of different categories 
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 217 unique categories.


In [99]:
# Now we analyse the neighborhoods

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postcode and neighborhood columns back to dataframe
toronto_onehot.drop('Neighborhood',axis=1,inplace=True) # the column exists in the middle of the df 
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move postcode, neighborhood column to the first column
fixed_columns = ['Postcode','Neighborhood']+ list(toronto_onehot.columns[:-2])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postcode,Neighborhood,Accessories Store,Airport,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,[Parkwoods],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M3A,[Parkwoods],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M4A,[Victoria Village],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [100]:
# check the size of the new dataframe
toronto_onehot.shape

(2579, 218)

In [101]:
# group rows by Postcode and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped

,Postcode,Accessories Store,Airport,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Auditorium,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1C,0.000000,0.0,0.010000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.010000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.00,0.02000,0.00,0.020000,0.010000,0.000000,0.000000,0.000000,0.0,0.040000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000,0.060000,0.000000,0.080000,0.000000,0.01,0.00,0.000000,0.010000,0.000000,0.00,0.000000,0.040000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.020000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.01,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00000,0.010000,0.000000,0.010000,0.00000,0.00,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.01,0.00,0.010000,0.00,0.00,0.000000

In [102]:
# check the size of the df
toronto_grouped.shape

(100, 217)

In [103]:
# print each neighborhood's postcode along with the top 5 most common venues
num_top_venues = 10

for pc in toronto_grouped['Postcode']:
    print("----"+pc+"----")
    temp = toronto_grouped[toronto_grouped['Postcode'] == pc].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1C----
                       venue  freq
0                Coffee Shop  0.08
1             Clothing Store  0.06
2             Cosmetics Shop  0.04
3                       Café  0.04
4  Middle Eastern Restaurant  0.03
5                      Diner  0.02
6                    Theater  0.02
7                 Restaurant  0.02
8                      Plaza  0.02
9                Pizza Place  0.02


----M1E----
                       venue  freq
0                Golf Course   1.0
1          Accessories Store   0.0
2              Moving Target   0.0
3             Medical Center   0.0
4   Mediterranean Restaurant   0.0
5                Men's Store   0.0
6              Metro Station   0.0
7         Mexican Restaurant   0.0
8  Middle Eastern Restaurant   0.0
9         Miscellaneous Shop   0.0


----M1G----
                        venue  freq
0                         Bar   1.0
1           Accessories Store   0.0
2              Medical Center   0.0
3    Mediterranean Restaurant   0.0
4         

In [104]:
# now the information is put into a pandas dataframe
#
# create a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [132]:
# create the new dataframe and display the top venues for each neighborhood' postcode

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1C,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Diner,Tea Room,Ramen Restaurant,Bookstore,Plaza
1,M1E,Golf Course,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run
2,M1G,Bar,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop
3,M1H,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Bakery,Cocktail Bar,Beer Bar,Breakfast Spot,Cosmetics Shop
4,M1J,Breakfast Spot,Pizza Place,Electronics Store,Medical Center,Mexican Restaurant,Rental Car Location,Intersection,Dessert Shop,Empanada Restaurant,Eastern European Restaurant


In [133]:
# run k-means to cluster the neighborhood into n clusters.

# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 3, 1, 6, 6, 6, 6, 0, 6, 6], dtype=int32)

In [134]:
# create a new dataframe that includes the cluster as well as the top venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_clean_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')


toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,[Parkwoods],43.753259,-79.329656,7.0,Park,Food & Drink Shop,Yoga Studio,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Dog Run
1,M4A,North York,[Victoria Village],43.725882,-79.315572,6.0,Coffee Shop,Hockey Arena,Portuguese Restaurant,French Restaurant,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
2,M5A,Downtown Toronto,[Harbourfront],43.654260,-79.360636,6.0,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Café,Restaurant,Breakfast Spot,Event Space,Electronics Store
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]",43.718518,-79.464763,6.0,Clothing Store,Accessories Store,Boutique,Fraternity House,Miscellaneous Shop,Furniture / Home Store,Coffee Shop,Carpet Store,Gift Shop,Shoe Store
4,M7A,Downtown Toronto,[Queen's Park],43.718518,-79.464763,6.0,Clothing Store,Accessories Store,Boutique,Fraternity House,Miscellaneous Shop,Furniture / Home Store,Coffee Shop,Carpet Store,Gift Shop,Shoe Store


In [144]:
# Some areas did not return a Cluster Laber, those will be grouped under Cluster number kclusters+1
toronto_merged.fillna(kclusters+1, inplace=True)

In [142]:
# Finally create a map with the clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters+1)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postcode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters